In [467]:
import warnings 
warnings.filterwarnings('ignore')

In [468]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from scipy.stats import zscore

In [469]:
# calculate accuracy measures and confusion matrix
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score, auc
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [470]:
df = pd.read_csv('Fine_2014_2019_Finger_millet_Min_Blore_Mysore.csv')
df.head()

,district,commodity,Arrival Date,variety,Min_price
0,Mysore,Ragi (Finger Millet),02-01-14,Fine,1600
1,Mysore,Ragi (Finger Millet),03-01-14,Fine,2000
2,Mysore,Ragi (Finger Millet),04-01-14,Fine,1800
3,Mysore,Ragi (Finger Millet),07-01-14,Fine,2000
4,Mysore,Ragi (Finger Millet),08-01-14,Fine,2100


In [471]:
gp = pd.pivot_table(df,values='Min_price',columns=['district','commodity'],index=['Arrival Date'])

In [472]:
gp.head()

district,Bangalore,Mysore
commodity,Ragi (Finger Millet),Ragi (Finger Millet)
Arrival Date,,
01-01-15,1900,1900
01-01-16,1800,1800
01-01-18,2800,2800
01-01-19,2800,2800
01-02-15,1900,1900


In [473]:
flattened = pd.DataFrame(gp.to_records())
flattened['Arrival Date'] = pd.to_datetime(flattened['Arrival Date'])
flattened = flattened.sort_values(by=['Arrival Date'],ascending=True)
df = flattened.reset_index()
del df['index']
df.head()

,Arrival Date,"('Bangalore', 'Ragi (Finger Millet)')","('Mysore', 'Ragi (Finger Millet)')"
0,2014-01-03,1800,1800
1,2014-01-04,1800,1800
2,2014-01-06,1800,1800
3,2014-01-07,1700,1700
4,2014-01-08,1850,1850


In [474]:
df1 = df.set_index('Arrival Date').asfreq('D')

In [475]:
df1.head()

,"('Bangalore', 'Ragi (Finger Millet)')","('Mysore', 'Ragi (Finger Millet)')"
Arrival Date,,
2014-01-03,1800.0,1800.0
2014-01-04,1800.0,1800.0
2014-01-05,NaN,NaN
2014-01-06,1800.0,1800.0
2014-01-07,1700.0,1700.0


In [476]:
ndf = df1.fillna(method='ffill')

In [477]:
ndf.head()

,"('Bangalore', 'Ragi (Finger Millet)')","('Mysore', 'Ragi (Finger Millet)')"
Arrival Date,,
2014-01-03,1800.0,1800.0
2014-01-04,1800.0,1800.0
2014-01-05,1800.0,1800.0
2014-01-06,1800.0,1800.0
2014-01-07,1700.0,1700.0


In [495]:
ndf.to_csv('PreProcessed_Fine_2014_2019_Finger_millet_Min_Blore_Mysore.csv')

In [496]:
df = pd.read_csv('PreProcessed_Fine_2014_2019_Finger_millet_Min_Blore_Mysore.csv')

In [497]:
df.head()

,Arrival Date,"('Bangalore', 'Ragi (Finger Millet)')","('Mysore', 'Ragi (Finger Millet)')"
0,2014-01-03,1800.0,1800.0
1,2014-01-04,1800.0,1800.0
2,2014-01-05,1800.0,1800.0
3,2014-01-06,1800.0,1800.0
4,2014-01-07,1700.0,1700.0


In [498]:
df['Arrival Date'] = pd.to_datetime(df['Arrival Date']).astype('int')

df.describe()

,Arrival Date,"('Bangalore', 'Ragi (Finger Millet)')","('Mysore', 'Ragi (Finger Millet)')"
count,2.044000e+03,2044.00000,2044.00000
mean,1.476965e+18,2406.38454,2406.38454
std,5.099296e+16,566.48057,566.48057
min,1.388707e+18,1400.00000,1400.00000
25%,1.432836e+18,1900.00000,1900.00000
50%,1.476965e+18,2200.00000,2200.00000
75%,1.521094e+18,3000.00000,3000.00000
max,1.565222e+18,3800.00000,3800.00000


In [499]:
X = df['Arrival Date'].values

X = X.reshape(-1,1)

In [500]:
y = df.iloc[:,[1]].values
y

array([[1800.],
       [1800.],
       [1800.],
       ...,
       [3100.],
       [3100.],
       [3100.]])

In [501]:

scaler = MinMaxScaler(feature_range=(0, 1))
y = scaler.fit_transform(y)

y

array([[0.16666667],
       [0.16666667],
       [0.16666667],
       ...,
       [0.70833333],
       [0.70833333],
       [0.70833333]])

In [502]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20, shuffle = False)

In [503]:
y_train

array([[0.16666667],
       [0.16666667],
       [0.16666667],
       ...,
       [0.66666667],
       [0.66666667],
       [0.66666667]])

In [504]:
from sklearn.neighbors import KNeighborsClassifier


In [505]:
nn = KNeighborsClassifier(n_neighbors=1)

In [512]:
from sklearn import preprocessing
from sklearn import utils

lab_enc = preprocessing.LabelEncoder()
y_train = lab_enc.fit_transform(y_train)
y_test = lab_enc.fit_transform(y_test)

In [513]:

knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [514]:
pred = knn.predict(X_test)

In [515]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import cross_val_score

In [520]:
print(confusion_matrix(pred,y_test))

[[  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  5   6  23  26 221  45  83   0]]


In [521]:
print(classification_report(pred,y_test))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           1       0.00      0.00      0.00       0.0
           2       0.00      0.00      0.00       0.0
           3       0.00      0.00      0.00       0.0
           4       0.00      0.00      0.00       0.0
           5       0.00      0.00      0.00       0.0
           6       0.00      0.00      0.00       0.0
          17       0.00      0.00      0.00     409.0

    accuracy                           0.00     409.0
   macro avg       0.00      0.00      0.00     409.0
weighted avg       0.00      0.00      0.00     409.0

